In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# To do:
 - Move path definition function to coco help
 - Comment path def function
 - Cull imports
 - make a separate train/test data import, modify training process
 - define our own train_one_epoch function
 - start saving out detection results after some n epochs for testing after long overnight run
 - define function for testing model
 - comment test function
 - move test function externally
 - make a function to load a model based on the save path used
 - investigate what the weight_decay does to the lr over time

In [2]:
# Make sure you have the right version of numpy or pycocotools won't work
! pip install numpy==1.17.5

     |████████████████████████████████| 20.0MB 1.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [0]:
import sys
notebook_folder = '/content/drive/My Drive/Colab Notebooks/xview_18/'
sys.path.append(notebook_folder)

## Subchip Images: Save them locally 

In [2]:
from coco_help import subchip_images_colab, convert_imgs_rgb

gt = notebook_folder + 'xView_train_18.json'
images = '/content/drive/My Drive/xView/train_images/'
new_images = '/content/train_images_400/'
chip_size = 400

#12051

chip_gt = subchip_images_colab(gt, images, new_images, chip_size)
convert_imgs_rgb(new_images)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


14828 annotations originally
846 images discovered in folders
50 images processed
100 images processed
150 images processed
200 images processed
250 images processed
300 images processed
350 images processed
400 images processed
450 images processed
500 images processed
550 images processed
12051 annotations saved
New ground truth: /content/drive/My Drive/Colab Notebooks/xview_18/xView_train_18_400.json
New images: /content/train_images_400/
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250


## Load Training Data

In [5]:
from pytorch_coco_detect import make_data_loader
import torch

train_data_dir = new_images
num_workers = 0
train_batch_size = 10

train_data_loader = make_data_loader(train_data_dir, chip_gt, train_batch_size, num_workers)

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


# Name Your Model
 - If you've already trained a model with these params, name it based on that and load previous work

In [6]:
from pytorch_coco_detect import define_model_path

model_folder ='/content/drive/My Drive/xView/'
num_classes = 19
model_name = 'resnet50fpn'
data_name = 'xview400'
optim = 'SGD'
lr = 0.005
mom = 0.9
wd = 0.0005
pretrained = False

path = define_model_path(model_folder, num_classes, model_name, data_name, optim, lr, mom, wd, pretrained, train_batch_size)
path

'/content/drive/My Drive/xView/resnet50fpn/xview400_classes_19_optim_SGD_lr_0p005_mom_0p9_wd_0p0005_pretrained_False_batch_10_epochs_16.pt'

# Load A Model

In [0]:
from pytorch_coco_detect import get_model_instance_segmentation

model = get_model_instance_segmentation(num_classes)

# move model to the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=lr, momentum=mom, weight_decay=wd)

# If the model isn't a new config, load previous parameters
epoch = int(path.split('_')[-1].replace('.pt', ''))
if epoch > 0:
    print(f'Loading {epoch} epochs of data')
    model.load_state_dict(torch.load(path)) 

# Train the model however many epochs

In [10]:
from pytorch_coco_detect import train_one_epoch

num_epochs = 2

for epoch in range(num_epochs):
    print(f'Epoch {epoch}')
    model, path, optimizer = train_one_epoch(model, train_data_loader, optimizer, device, path)

path

Epoch 0
Iteration: 0/440, Loss: 0.23435282707214355
Iteration: 25/440, Loss: 0.15810167789459229
Iteration: 50/440, Loss: 0.13785713911056519
Iteration: 75/440, Loss: 0.15758509933948517
Iteration: 100/440, Loss: 0.10901767015457153
Iteration: 125/440, Loss: 0.20267269015312195
Iteration: 150/440, Loss: 0.1291400045156479
Iteration: 175/440, Loss: 0.16088922321796417
Iteration: 200/440, Loss: 0.1631724089384079
Iteration: 225/440, Loss: 0.19088497757911682
Iteration: 250/440, Loss: 0.1379162073135376
Iteration: 275/440, Loss: 0.1508105993270874
Iteration: 300/440, Loss: 0.10724844038486481
Iteration: 325/440, Loss: 0.11438004672527313
Iteration: 350/440, Loss: 0.1626952737569809
Iteration: 375/440, Loss: 0.16390836238861084
Iteration: 400/440, Loss: 0.20472452044487
Iteration: 425/440, Loss: 0.1330810934305191
Epoch 1
Iteration: 0/440, Loss: 0.21902379393577576
Iteration: 25/440, Loss: 0.17715469002723694
Iteration: 50/440, Loss: 0.11368414014577866
Iteration: 75/440, Loss: 0.124202497

'/content/drive/My Drive/xView/resnet50fpn/xview400_classes_19_optim_SGD_lr_0p005_mom_0p9_wd_0p0005_pretrained_False_batch_10_epochs_18.pt'

## Test the finished model's performance

In [14]:
# If the model isn't a new config, load previous parameters
model.load_state_dict(torch.load(path))

<All keys matched successfully>

# Load the data you would like to test the model over, with a batch size of 1

In [24]:
from pytorch_coco_detect import make_data_loader

test_gt = chip_gt
test_data_dir = new_images
num_workers = 0
test_batch_size = 1 # Keep this the same

test_data_loader = make_data_loader(test_data_dir, test_gt, test_batch_size, num_workers)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [0]:
from pytorch_coco_detect import detector_dts

In [0]:
import torchvision
def detector_dts(model, data_loader, dt_path, iuo_nms):
    '''
    Purpose: Test a loaded model on a given data set
    TODO: make a better save file name to keep track of model/results relationships
    '''
    # Establish device settings and set model in appropriate mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    model.eval()

    detections = []
    i = 0

    # Evaluate all images
    with torch.no_grad():
        for imgs, annotations in data_loader:
            # Get data ready to evaluate
            imgs = list(img.to(device) for img in imgs)
            annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

            # Return detections
            d = model(imgs)

            # Pull this image's image ID
            im_id = int(annotations[0]['image_id'])

            # Perform non-max suppression to remove overlapping detections
            nms_indices = torchvision.ops.nms(d[0]['boxes'], d[0]['scores'], iuo_nms).tolist()

            # Get boxes, labels, scores 
            boxes = d[0]['boxes'].tolist()
            labels = d[0]['labels'].tolist()
            scores = d[0]['scores'].tolist()

            # Only process results which survive nms
            for a in nms_indices:
                # re-factor bbox and get detection area
                bbox = boxes[a]
                new_bbox = [int(bbox[0]), int(bbox[1]), int(bbox[2] - bbox[0]), int(bbox[3] - bbox[1])]
                area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

                # Create coco-style detection
                detection = {'id': i, 'image_id': im_id,
                            'category_id': labels[a], 'score' : scores[a],
                            'bbox': new_bbox, 'area': area, 'iscrowd': 0}
                detections.append(detection)
                i += 1   

    # Save out detections
    if os.path.exists(dt_path):
        os.remove(dt_path)  
    with open(dt_path, 'w') as f:
        json.dump(detections, f) 

    return

In [0]:
dt_path = path.replace('.pt', '_dts.json')
iou = 0.4

detector_dts(model, test_data_loader, dt_path, iou)

In [0]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

ann_type = 'bbox'
coco_gt_path ='/content/drive/My Drive/Colab Notebooks/xview_18/xView_train_18_400.json'
dt_path = '/content/drive/My Drive/Colab Notebooks/xview_18/xView_train_18_400_dt.json'
coco_gt = COCO(coco_gt_path)
coco_dt = coco_gt.loadRes(dt_path)

types = []
with open(coco_gt_path, 'r') as f:
    gts = json.load(f)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.34s)
creating index...
index created!


In [0]:
# running evaluation
cocoEval = COCOeval(coco_gt,coco_dt,ann_type)
# cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=12.10s).
Accumulating evaluation results...
DONE (t=1.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.305
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.071
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.126
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.045
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.178
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.198
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.162
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10